# THIS NOTEBOOK CAN BE USED TO COMPARE WVFs FROM ANY SOURCE
### 1. Load the WVFs from the source. Note the following variables to load the data:
    - pretrigger: the number of samples before the peak. Wvfs are automatically aligned to the peak.
    - sampling: the sampling rate of the data.
    - convert_sampling: None or a number. If a number, the data will be resampled to this sampling rate.
    - same_length: True or False. If True, all WVFs will be truncated or padded to the wvf_length. If False, the WVFs will be plotted as is.
    - wvf_length: desired length of the wvf. If the wvf is shorter than this, it will be padded with zeros. If it is longer, it will be truncated.

In [4]:
import sys; sys.path.insert(0, '../'); from lib.__init__ import *

pretrigger = 100
wvf_length = 1000

ir02_wvfs = import_templates(template_path="wvfs/IR02/",these_types=["LASER","SPE"],pretrigger=pretrigger,sampling=4e-9,convert_sampling=None,same_length=True,wvf_length=wvf_length,debug=False)
larsoft_wvfs = import_templates(template_path="wvfs/IR02/",these_types=["TEMPLATE"],pretrigger=pretrigger,sampling=16e-9,convert_sampling=True,new_sampling=4e-9,same_length=True,wvf_length=wvf_length,debug=False)

wvfs = ir02_wvfs + larsoft_wvfs

Loaded template FBK OV 1 int 19783.011863
Loaded template FBK OV 3 int 74425.841641
Loaded template FBK OV 2 int 30459.408217
Loaded template FBK OV 2 int 1448.816596
Loaded template FBK OV 3 int 3326.579613
Loaded template FBK OV 1 int 970.860736
Loaded template HPK OV 2 int 54762.613977
Loaded template HPK OV 1 int 38144.688526
Loaded template HPK OV 3 int 72899.759776
Loaded template HPK OV 2 int 733.928312
Loaded template HPK OV 1 int 595.800984
Loaded template HPK OV 3 int 872.080161
Loaded template FBK OV 2 int 1308.286721
Loaded template HPK OV 2 int 745.577355


### 2. Combine the WVFs into a single dataframe.

In [5]:
df = pd.DataFrame(wvfs)
display(df)

,INST,NAME,MODEL,TYPE,OV,SAMPING,AMP,INT,ADC,TIME
0,CIEMAT,FBK_OV1_run54_Laser_SC.root,FBK,LASER,1,4.000000e-09,195.826520,19783.011863,"[0.11429667519182171, 0.11706734867860773, 0.1...","[0.0, 4e-09, 8e-09, 1.2000000000000002e-08, 1...."
1,CIEMAT,FBK_OV3_run56_Laser_SC.root,FBK,LASER,3,4.000000e-09,711.141739,74425.841641,"[0.09753888859232138, 0.11467442481182706, 0.0...","[0.0, 4e-09, 8e-09, 1.2000000000000002e-08, 1...."
2,CIEMAT,FBK_OV2_run55_Laser_SC.root,FBK,LASER,2,4.000000e-09,294.549431,30459.408217,"[0.027981315436235363, 0.05466587919462463, 0....","[0.0, 4e-09, 8e-09, 1.2000000000000002e-08, 1...."
3,CIEMAT,FBK_OV2_SPE_SC.root,FBK,SPE,2,4.000000e-09,12.662743,1448.816596,"[0.22435114303691256, 0.21181968330536893, 0.2...","[0.0, 4e-09, 8e-09, 1.2000000000000002e-08, 1...."
4,CIEMAT,FBK_OV3_SPE_SC.root,FBK,SPE,3,4.000000e-09,30.156535,3326.579613,"[-0.21928228529225763, -0.30875263768727773, -...","[0.0, 4e-09, 8e-09, 1.2000000000000002e-08, 1...."
5,CIEMAT,FBK_OV1_SPE_SC.root,FBK,SPE,1,4.000000e-09,8.366773,970.860736,"[-0.032170423050775256, -0.09534042775714978, ...","[0.0, 4e-09, 8e-09, 1.2000000000000002e-08, 1...."
6,CIEMAT,HPK_OV2_run10_AveWvfLaser.npz,HPK,LASER,2,4.000000e-09,633.963263,54762.613977,"[0.02071456158699592, -0.07761477281271971, -0...","[0.0, 4e-09, 8e-09, 1.2000000000000002e-08, 1...."
7,CIEMAT,HPK_OV1_run09_AveWvfLaser.npz,HPK,LASER,1,4.000000e-09,443.500787,38144.688526,"[0.08765248070600157, -0.0157866022052221, 0.0...","[0.0, 4e-09, 8e-09, 1.2000000000000002e-08, 1...."
8,CIEMAT,HPK_OV3_run11_AveWvfLaser.npz,HPK,LASER,3,4.000000e-09,841.246426,72899.759776,"[0.0597205616963683, -0.050230562448304227, 0....","[0.0, 4e-09, 8e-09, 1.2000000000000002e-08, 1...."
9,CIEMAT,HPK_OV2_run02_AveWvfSPE.npz,HPK,SPE,2,4.000000e-09,8.631962,733.928312,"[0.08426396000871578, -0.09736921221209317, -0...","[0.0, 4e-09, 8e-09, 1.2000000000000002e-08, 1...."


### 3. Plot the WVFs.

In [6]:
this_df = df.explode(["ADC","TIME"])
fig = px.line(this_df,
                x="TIME",
                y="ADC",
                color="OV",
                facet_col="MODEL",
                facet_row="TYPE",
                title="CIEMAT Templates")
fig = format_coustom_plotly(fig,figsize=(None,600))
fig.update_yaxes(matches=None)
fig.show()

### 4. Enjoy!